# **Imports**

In [1]:
import pandas as pd
from openpyxl import load_workbook
from dataclasses import dataclass
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
from pydantic import BaseModel, ConfigDict


# **Classes**

In [2]:
class AdressDescription(BaseModel):
    adress: str
    model_config = ConfigDict(arbitrary_types_allowed=True)
    UNOM_ID: int
    EXOGEN_ID: int
    building_characteristics: pd.DataFrame
    events: pd.DataFrame
    anomaly_ts_2023: pd.DataFrame
    anomaly_ts_2024: pd.DataFrame
    

# **Constants**

In [3]:
@dataclass(frozen=True)
class Pathes:
    path_adresses_building_characteristics: str = '../main_datasets/processed_datasets/14. ВАО_Многоквартирные_дома_с_технико_экономическими_характеристиками__ВАО.csv' 
    path_turn_on_off_heating: str = '../main_datasets/processed_datasets/6. Плановые-Внеплановые отключения 01.10.2023-30.04.2023.csv'
    path_events_for_period_2023_params: str = '../main_datasets/processed_datasets/События за период_01.10.2023-31.12.2023____pivot_table.csv'
    path_events_for_period_2023: str = '../main_datasets/processed_datasets/События за период_01.10.2023-31.12.2023.xlsx____uploading_table.csv'
    path_events_for_period_2024: str = '../main_datasets/processed_datasets/События_за_период_01.01.2024-30.04.2024____uploading_table.csv'
    path_events_for_period_2024_params: str = '../main_datasets/processed_datasets/События_за_период_01.01.2024-30.04.2024____pivot_table.csv'
    path_to_timeline: str = '../variables/timeline'

In [4]:
date_columns =  [
    'Дата создания во внешней системе', 'Дата закрытия', 'Дата и время завершения события во внешней системе',
    'Дата регистрации отключения', 'Планируемая дата отключения', 'Планируемая дата включения', 'Фактическая дата отключения'
    'Фактическая дата включения'
]

In [5]:
usfull_building_params = [
    'Серии проектов', 'Количество этажей', 'Количество подъездов', 'Количество квартир',
    'Общая площадь', 'Общая площадь жилых помещений',
    'Общая площадь нежилых помещений', 'Износ объекта (по БТИ)',
    ' Материалы стен', 'Признак аварийности здания',
    'Количество пассажирских лифтов', 'Количество грузопассажирских лифтов',
    'Очередность уборки кровли', 'Материалы кровли по БТИ',
    'Типы жилищного фонда', 'Статусы МКД'
]

# **Functions**

In [6]:
def date_columns_to_date_format(df: pd.DataFrame) -> pd.DataFrame:
    for i in date_columns:
        if i in df.columns:
            df[i] = pd.to_datetime(df[i])
    return df

In [7]:
def load_dataset(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    df = date_columns_to_date_format(df)
    return df

In [8]:
def make_timeline_for_year(year: int=2023):
    data_1 = np.arange(datetime(year=year, month=1, day=1), datetime(year=year, month=5, day=1), timedelta(hours=1)).tolist()
    data_2 = np.arange(datetime(year=year, month=10, day=1), datetime(year=year+1, month=1, day=1), timedelta(hours=1)).tolist()
    return np.array(data_1+data_2)


def make_timeline_for_period(year: int=2022):
    s = datetime(year=year, month=10, day=1)
    e = datetime(year=year+1, month=5, day=1)
    data = np.arange(s, e, timedelta(hours=1)).tolist()
    return data

In [9]:
def address_compare(a1_BTI, a2_BTI):
    a1_BTI = a1_BTI.replace('город Москва, ', '').lower().split(', ')[1:]
    a2_BTI = a2_BTI.replace('город Москва, ', '').lower().split(', ')
    for a1_i, a2_i in zip(a1_BTI, a2_BTI):
        if a1_i != a2_i:
            return False
    return True

    

In [10]:
def make_anomaly_ts(year: int, events: pd.DataFrame):
    time_line = make_timeline_for_period(year)
    anomaly = [any([i.month == j.month and i.day == j.day and i.hour == j.hour for j in events['Дата создания во внешней системе']]) for i in time_line]
    nd = pd.DataFrame({'time': time_line, 'anomaly': anomaly})
    return nd
    

In [11]:
def get_address_description(adress, events_df, addres_df):
    building_characteristic = addres_df[adress == addres_df['Адрес по БТИ']]

    UNOM = building_characteristic['УНОМ'].values[0]
    EXOGEN_ID = building_characteristic['Идентификатор из сторонней системы'].values[0]

    building_characteristic = building_characteristic.loc[:, usfull_building_params]
    events = events_df[UNOM == events_df['УНОМ']]

    time_line_2023 = make_timeline_for_period(2022)
    time_line_2024 = make_timeline_for_period(2023)

    anomaly_2023 = [any([i.month == j.month and i.day == j.day and i.hour == j.hour for j in events['Дата создания во внешней системе']]) for i in time_line_2023]
    anomaly_2024 = [any([i.month == j.month and i.day == j.day and i.hour == j.hour for j in events['Дата создания во внешней системе']]) for i in time_line_2024]
    d2023 = pd.DataFrame({'time': time_line_2023, 'anomaly': anomaly_2023})
    d2024 = pd.DataFrame({'time': time_line_2024, 'anomaly': anomaly_2024})
    return AdressDescription(
        adress=adress,
        UNOM_ID=UNOM,
        EXOGEN_ID=EXOGEN_ID,
        building_characteristics=building_characteristic,
        events=events,
        anomaly_ts_2023=d2023,
        anomaly_ts_2024=d2024,
    )

# **Loading tables**

In [12]:
turn_on_off_heating = load_dataset(Pathes.path_turn_on_off_heating)
adresses_building_characteristics = load_dataset(Pathes.path_adresses_building_characteristics)
events_for_period_2023 = load_dataset(Pathes.path_events_for_period_2023)
events_for_period_2024 = load_dataset(Pathes.path_events_for_period_2024)
events_for_period_2023_params = load_dataset(Pathes.path_events_for_period_2023_params)
events_for_period_2024_params = load_dataset(Pathes.path_events_for_period_2024_params)


In [13]:
adresses_building_characteristics.head()

,Адрес из сторонней системы,Идентификатор из сторонней системы,Адрес по БТИ,УНОМ,Округ,Район,Серии проектов,Количество этажей,Количество подъездов,Количество квартир,...,Общая площадь нежилых помещений,Износ объекта (по БТИ),Материалы стен,Признак аварийности здания,Количество пассажирских лифтов,Количество грузопассажирских лифтов,Очередность уборки кровли,Материалы кровли по БТИ,Типы жилищного фонда,Статусы МКД
0,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Но...",1520521,"город Москва, Суздальская улица, дом 24, корпус 2",31783,Восточный административный округ,муниципальный округ Новокосино,П-55,14,3.0,130.0,...,595.6,NaN,панельные,нет,6.0,0.0,NaN,прочая(черепица;щепа;дранка),МКД,в эксплуатации
1,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Со...",1526496,"город Москва, улица Сокольнический Вал, дом 4",24231,Восточный административный округ,муниципальный округ Сокольники,нет данных,5,3.0,61.0,...,203.6,NaN,из железобетонных сегментов,нет,0.0,0.0,1 очередь,мягкая-совмещенная с рубероидным покрытием,МКД,в эксплуатации
2,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Но...",1502303,"город Москва, Новокосинская улица, дом 38, кор...",16560,Восточный административный округ,муниципальный округ Новокосино,П-44,17,2.0,128.0,...,2547.5,NaN,панельные,нет,2.0,2.0,NaN,мягкая-совмещенная с рубероидным покрытием,МКД,в эксплуатации
3,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Ив...",1502774,"город Москва, Малый Купавенский проезд, дом 1",12180,Восточный административный округ,муниципальный округ Ивановское,П-68,16,2.0,206.0,...,1021.8,NaN,железобетонные,нет,2.0,2.0,NaN,мягкая-совмещенная с рубероидным покрытием,МКД,в эксплуатации
4,"Дом по адресу РФ, г. Москва, вн.тер.г. м.о. Из...",1502277,"город Москва, 5-я Парковая улица, дом 52",18617,Восточный административный округ,муниципальный округ Измайлово,нет данных,5,9.0,155.0,...,2208,NaN,кирпичные,нет,0.0,0.0,1 очередь,асбофанера-шифер,МКД,в эксплуатации


In [14]:
adresses_building_characteristics['Адрес по БТИ'].map(lambda a: 'Зборовский переулок' in a and 'дом 11' in a).sum()

1

In [15]:
adresses_building_characteristics['Адрес по БТИ'][0]

'город Москва, Суздальская улица, дом 24, корпус 2'

In [16]:
usefull_events = events_for_period_2024_params['Названия строк'].to_list()

In [17]:
events_for_period_2023__only_usefull_events = events_for_period_2023.loc[events_for_period_2023['Наименование'].isin(usefull_events)]
events_for_period_2024__only_usefull_events = events_for_period_2024.loc[events_for_period_2024['Наименование'].isin(usefull_events)]

In [18]:
df = events_for_period_2024__only_usefull_events
df.columns = events_for_period_2023__only_usefull_events.columns
all_in_one_events = pd.concat(
    [
        events_for_period_2023__only_usefull_events, 
        df
    ],
    axis=0
)

In [19]:
events_for_period_2023__only_usefull_events['Адрес'].unique()[0].split(', ')

['внутригородская территория муниципальный округ Преображенское',
 '1-й Зборовский переулок',
 'дом 11']

# **Make datasets**

In [20]:
all_adresses = adresses_building_characteristics['Адрес по БТИ'].unique()

In [21]:
from tqdm import tqdm
import pickle
import os

In [22]:
created_files = os.listdir('../main_datasets/pickle_ts_anomalies/')

In [23]:
for i in tqdm(all_adresses):
    x = get_address_description(
        i, 
        events_df=all_in_one_events, 
        addres_df=adresses_building_characteristics
    )
    fn = i.replace('/', '-').replace(',', '').replace('.', '').replace(' ', '_')
    asd = '{fn}.pkl'
    if not asd in created_files:
        with open(f'../main_datasets/pickle_ts_anomalies/{fn}.pkl', 'wb') as f:
            pickle.dump(x, f)
    

  0%|          | 0/4368 [00:00<?, ?it/s]

100%|██████████| 4368/4368 [21:06<00:00,  3.45it/s]
